In [1]:
import math

In [2]:
def computeconcreteshearstrength(vctype, lamda, fcprime, width, depth, vu, mu, nu, db, ds, cover):

    height = depth + db/2 + ds + cover
    ag = height * width
    rhopw = numbars * math.pi * math.pow((db/2),2) / (width*depth)
    
    if vctype == "simple without axial":
        vc = (1.0 / 6.0) * lamda * math.sqrt(fcprime) * width * depth

    elif vctype == "detailed with vu and mu":
        if vu*depth/mu <1: # OR GREATER THAN 1??????? 
            vc1 == (0.16 * lamda * math.sqrt(fcprime) + 17 * rhopw * (vu * depth/ mu)) * width * depth
        else:
            vc1 == (0.16 * lamda * math.sqrt(fcprime) + 17 * rhopw) * width * depth
        vc2 = 0.29 * lamda * math.sqrt(fcprime) * width * depth
        vc = min(vc1, vc2)

    elif vctype == "simple with axial compression":
        vc = (1 / 6) * (1 + (nu / (14 * ag))) * lamda * math.sqrt(fcprime) * width * depth

    elif vctype == "detailed with axial compression":
        vc1 = 0.16 * lamda * math.sqrt(fcprime) + 17 * rhopw * (vu*depth/ (mu - nu * ((4 * height - depth) / 8))) * width * depth
        vc2 = 0.29 * lamda * math.sqrt(fcprime) * width * depth * math.sqrt(1 + (0.29 * nu / ag))
        if mu - nu * ((4 * height - depth) / 8) <= 0:
            vc = vc2
        else:
            vc = min (vc1, vc2)

    elif vctype == "simple with axial tension":
        vc = (1 / 6) * (1 + (0.29 * nu / ag)) * lamda * math.sqrt(fc) * width * depth

    return vc

In [3]:
def computesteelshearstrength(av, fyt, depth, stirrupspacing):
    vs = av * fyt * depth / stirrupspacing
    return vs

def computenominalshearstrength(vc, vs):
    vn = vc + vs
    return vn

def computeultimateshearstrength(vn, phi):
    vu = phi * vn
    return vu

##################################
# GIVEN STIRRUP SPACING => FIND VU
##################################

if __name__ == "__main__":
    
    vctype = "simple without axial"
                             # simple without axial
                             # detailed with vu and mu
                             # simple with axial compression
                             # detailed with axial compression
                             # simple with axial tension
    
    lamda = 1.0              # Normal weight concrete
    fcprime = 27.6          # MPa
    width = 300              # mm
    depth = 510              # mm
    phi = 0.75
    fyt = 413.7              # MPa
    stirrupspacing = 140     # mm
    numlegs = 2
    ds = 10                  # mm
    db = 25                  # mm
    numbars = 4             
    cover = 40
    vu = None
    mu = None
    nu = None               # negative for tension, positive for compression

    av = numlegs * math.pi * (ds / 2.0)**2  # mm^2
    vc_calculated = computeconcreteshearstrength(vctype, lamda, fcprime, width, depth, vu, mu, nu, db, ds, cover)
    vs_calculated = computesteelshearstrength(av, fyt, depth, stirrupspacing)
    vn_calculated = computenominalshearstrength(vc_calculated, vs_calculated)
    vu_calculated = computeultimateshearstrength(vn_calculated, phi)

    print("=== GIVEN STIRRUP SPACING => FIND VU ===")
    print(f"Given stirrup spacing = {stirrupspacing:.2f} mm")
    print(f"Ultimate shear = {vu_calculated / 1000:.2f} kN")

=== GIVEN STIRRUP SPACING => FIND VU ===
Given stirrup spacing = 140.00 mm
Ultimate shear = 278.02 kN


In [4]:
def computespacing(fcprime, width, fyt, depth, phi, vu_required):
    
    av = numlegs * math.pi * math.pow((ds / 2.0), 2)  # mm^2
    vc = computeconcreteshearstrength(vctype, lamda, fcprime, width, depth, vu, mu, nu, db, ds, cover)
    vs_required = (vu_required / phi) - vc
    vs_limit = (1.0 / 3.0) * math.sqrt(fcprime) * width * depth

    if vu_required > phi * vc:
        # Avreq calculation
        use_s = av * fyt * depth / vs_required
    else:
        # Avmin calculation
        avmin1 = 0.062 * math.sqrt(fcprime) * width / fyt
        avmin2 = 0.35 * width / fyt
        avmin_per_s = max(avmin1, avmin2)
        use_s = av / avmin_per_s

    if vs_required <= vs_limit:
        smax = min(depth / 2.0, 600.0)
    else:
        smax = min(depth / 4.0, 300.0)

    stirrupspacing = min(use_s, smax)
    return stirrupspacing

#########################################
# GIVEN VU => FIND REQUIRED STIRRUP SPACING
#########################################

if __name__ == "__main__":
    
    lamda = 1.0             # Normal weight concrete
    fcprime = 27.6         # MPa
    width = 300           # mm
    depth = 510           # mm
    phi = 0.75
    fyt = 413.7             # MPa
    vu_required = 278.02e3   # N
    numlegs = 2
    ds = 10
    db = 25                  # mm
    numbars = 4             
    cover = 40
    vu = None
    mu = None
    nu = None                # negative for tension, positive for compression

    required_spacing = computespacing(fcprime, width, fyt, depth, phi, vu_required)

    print("=== GIVEN VU => FIND REQUIRED STIRRUP SPACING ===")
    print(f"Given ultimate shear = {vu_required / 1000:.2f} kN")
    print(f"Required stirrup spacing = {required_spacing:.2f} mm")

=== GIVEN VU => FIND REQUIRED STIRRUP SPACING ===
Given ultimate shear = 278.02 kN
Required stirrup spacing = 140.00 mm
